# National Leagues Data

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import json

In [29]:
def url_league(league_name, season):
    return f"https://understat.com/league/{league_name}/{season}"

In [30]:
def get_data(league_name, season, i):
    # get the url
    url=url_league(league_name, season)

    # parse the html page
    dataLigue1=requests.get(url_league(league_name,str(season)), "html.parser")
    soupLigue1=BeautifulSoup(dataLigue1.text, "html.parser")

    # get the scripts
    scripts=soupLigue1.find_all('script')
    strings=scripts[i].string

    # Convert as JSON
    ind_start = strings.index("('")+2 
    ind_end = strings.index("')") 
    json_data = strings[ind_start:ind_end] 
    json_data = json_data.encode('utf8').decode('unicode_escape')

    # Convert as DataFrame
    df=pd.read_json(json_data)

    return df

In [34]:
# save the data
df_players = get_data("EPL", "2021", 3)
df_players.to_csv("PL_2021.csv")

# Scrap wiki

In [16]:
    import requests
    from bs4 import BeautifulSoup
    import pandas as pd

    from time import sleep
    from selenium import webdriver
    #from selenium.webdriver.common.keys import Keys
    #import json
    from tqdm import tqdm, tqdm_pandas
    tqdm_pandas(tqdm())
    import time 
    from time import sleep

0it [00:00, ?it/s]


In [40]:
#load the data

PL_2021 = pd.read_csv('PL_2021.csv')


In [36]:
# take a dico of the wiki data
# return a row with the data
def get_out_of_the_dico(dico,player_id):
    player_international = None 
    actual_team = None 
    number = None 
    Name = None 
    mois_de_naissance = None 
    Nationalite = None 
    jour_de_naissance = None 
    anne_de_naissance = None 
    age_actuel = None 
    taille_en_m = None 
    Poste = None 
    Pied_fort = None 
    if "Sélections" in dico.keys() :
        player_international = dico["Sélections"]
    if "Équipe" in dico.keys():
        actual_team = dico["Équipe"]
    if "Numéro"in dico.keys():
        number = dico["Numéro"]
    if "Nom" in dico.keys():
        Name = dico["Nom"]
    if "Nationalité"in dico.keys():
        Nationalite = dico["Nationalité"]
    if "Naissance" in dico.keys():
        mois_de_naissance = dico["Naissance"].split()[1]
        jour_de_naissance = dico["Naissance"].split()[0]
        anne_de_naissance = dico["Naissance"].split()[2]
        age_actuel = dico["Naissance"].split()[3][1:]
    if "Taille" in dico.keys():
        taille_en_m = dico["Taille"].split()[0]
    if "Poste" in dico.keys():
        Poste = dico['Poste']
    if "Pied fort" in dico.keys():
        Pied_fort = dico['Pied fort']
    return player_international, actual_team,number,Name,Nationalite,mois_de_naissance,jour_de_naissance,anne_de_naissance,age_actuel,taille_en_m,Poste,Pied_fort,player_id

In [37]:
# get the urlpage
# return dico with the data
def get_the_data_contains_on_the_page(htmlplayerpage):
    stopwords = ['']
    dico = {}
    dico['Sélections']=False
    # get all the data that we want 
    for item in htmlplayerpage.find("table",{"class":"infobox_v2 noarchive"}).findAll('tr'):
        text_item = item.getText()
        if text_item=='\nSélections en équipe nationale2':
            dico['Sélections']=True
        elif text_item!=None:
            text_item = text_item.split('\n')
            text_item = [word for word in text_item if word not in stopwords]
            if len(text_item)==2:
                dico[text_item[0]]=text_item[1]
    return dico 

In [38]:
# get a  player name
# return the data of this player after scarping wiki
def get_the_htlm_page_of_a_player(row):
    player_name = row["player_name"]
    id = row["id"]
    player_name_send = player_name
    base_url = "https://fr.wikipedia.org/wiki/"
    player_name = player_name.split()
    # get the name looks like what ask wiki 
    if len(player_name)==2:
        url = base_url+ player_name[0]+'_'+player_name[1]
    else:
        url = base_url+ player_name[0]
    # get the url data 
    request_data =requests.get(url)
    htmlplayerpage=BeautifulSoup(request_data.text, "html.parser")
    # try if that is the good url 
    try: 
        htmlplayerpage.find("table",{"class":"infobox_v2 noarchive"}).findAll('tr')
    except:
        return None
    else:
        # work on the data 
        dico =get_the_data_contains_on_the_page(htmlplayerpage)
        data = get_out_of_the_dico(dico,id)
        return data
    

In [42]:
data_scrap = PL_2021.progress_apply(get_the_htlm_page_of_a_player,axis=1)
filtered_list = [x for x in data_scrap if x is not None]

100%|██████████| 537/537 [10:58<00:00,  1.23s/it]


In [43]:
# create a df for the new data
def new_data(filtered_list,all_player_big_6):
    wiki_data= pd.DataFrame(columns=["player_international", "actual_team","number","Name","Nationalite","mois_de_naissance","jour_de_naissance","anne_de_naissance","age_actuel","taille_en_m","Poste","Pied_fort","id"])
    for i in filtered_list:
        wiki_data.loc[len(wiki_data)]=i
    final_data_set = wiki_data.merge(all_player_big_6,on= "id")
    return final_data_set

In [45]:
# save the new df
final_data_set = new_data(filtered_list,PL_2021)
final_data_set.to_csv("df_player_wiki_pl.csv")

In [1]:
from selenium import webdriver
driver =  webdriver.Firefox()

In [2]:
driver.get("https://www.fifaindex.com/")

WebDriverException: Message: Reached error page: about:neterror?e=dnsNotFound&u=https%3A//www.fifaindex.com/&c=UTF-8&d=Impossible%20de%20se%20connecter%20au%20serveur%20%C3%A0%20l%E2%80%99adresse%20www.fifaindex.com.
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:182:5
UnknownError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:509:5
checkReadyState@chrome://remote/content/marionette/navigate.sys.mjs:61:24
onNavigation@chrome://remote/content/marionette/navigate.sys.mjs:329:39
emit@resource://gre/modules/EventEmitter.sys.mjs:154:20
receiveMessage@chrome://remote/content/marionette/actors/MarionetteEventsParent.sys.mjs:35:25


# Scrap index fifa

In [10]:
#get url player page
#return dico with all the data
def return_a_dico_with_all_the_data(url_player):
    dico = {}
    request_data =requests.get(url_player)
    htmlplayerpage=BeautifulSoup(request_data.text, "html.parser")
    # get all the data we interst of 
    for item in htmlplayerpage.find("div",{"class":"masonry"}).findAll('p'):
        text_item = item.getText()
        if text_item!=None:
            text_item = text_item.split()
            if len(text_item)==2:
                dico[text_item[0]]=text_item[1]
    return dico

In [11]:
#player name and bool if this is the first time
#Return player name for the scaping
def get_correct_player_name_for_selenium(player_name_df,first):
    player_name_df_splip = player_name_df.split()
    if len(player_name_df_splip)>=1 and first==True:
        if len(player_name_df_splip)==2:
            return player_name_df
        else:
            return player_name_df_splip[-1]
    else:
        return player_name_df_splip[-1]

In [12]:
#get a player name
# return all the data
def get_the_url_player(row):
    dico ={}
    # get a player like the website ask
    player_name = get_correct_player_name_for_selenium(row['player_name'],True)
    # find the bar
    shearch_bar = driver.find_element('xpath', '//*[@id="site-search"]')
    shearch_bar.click()
    # rigth a player name 
    shearch_bar.send_keys(player_name)
    sleep(1.1)
    # try to get the data 
    try:
        driver.find_element("xpath",'/html/body/header/nav/div/form/span/div/div/div[1]/div')
    except:
        # try again with a other name 
        driver.get("https://www.fifaindex.com/")
        player_name = get_correct_player_name_for_selenium(row['player_name'],False)
        shearch_bar = driver.find_element('xpath', '//*[@id="site-search"]')
        shearch_bar.click()
        shearch_bar.send_keys(player_name)
        sleep(2)
        try: 
            driver.find_element("xpath",'/html/body/header/nav/div/form/span/div/div/div[1]/div')
        except:
            return None
        else:
            first_player = driver.find_element("xpath",'/html/body/header/nav/div/form/span/div/div/div[1]/div')
            first_player.click()
            sleep(0.2)
            url_player = driver.current_url
            dico = return_a_dico_with_all_the_data(url_player)
            dico['id'] = row['id']
    else:
        # get the url of the player 
        first_player = driver.find_element("xpath",'/html/body/header/nav/div/form/span/div/div/div[1]/div')
        first_player.click()
        sleep(0.2)
        url_player = driver.current_url
        dico = return_a_dico_with_all_the_data(url_player)
    return dico 


In [35]:
df = pd.read_csv('df_player_wiki_fifa_pl.csv')

In [43]:
type(df['dico_fifa'][400])

float

In [46]:
df.shape

(426, 42)

In [52]:
for index,row in tqdm(df.iterrows()):
        if type(row['dico_fifa']) == float:
                print(index)
                row["dico_fifa"] = get_the_url_player(row)
                if row["dico_fifa"] == None:
                        driver.get("https://www.fifaindex.com/")


                df['dico_fifa'].at[index] = row['dico_fifa']

0


21


285


301


303


356


425


426it [01:46,  3.99it/s]


In [50]:
df.to_csv('df_player_wiki_fifa_pl.csv')

In [36]:
df_with_new_col = pd.DataFrame(columns=["Dribbling","Marking","Aggression" ,"Reactions" ,"Interceptions" ,"Vision" ,"Composure" ,"Crossing" ,"Acceleration" ,"Strength" ,"Balance" ,"Agility","Penalties" ,"Volleys" ,"Power" ,"Jumping" ,"Heading","Curve","Injury","id","Finishing"])

In [56]:
import sys, ast, types

In [60]:
# get a dico 
# return the data 
def get_out_of_the_dico(dico,player_id):
    if type(dico)!= float: 
        try:
            ast.literal_eval(str(dico))
        except:
            return None
        else:
            dico = ast.literal_eval(str(dico))
            Dribbling = None 
            Marking = None 
            Aggression = None 
            Reactions = None 
            Interceptions = None 
            Vision = None 
            Composure = None 
            Crossing = None 
            Acceleration = None 
            Strength = None 
            Balance = None 
            Agility = None
            Penalties = None 
            Volleys = None 
            Power = None 
            Jumping = None 
            Heading = None 
            Curve = None
            Injury = None 
            Finishing = None
            if "Dribbling" in dico.keys() :
                Dribbling = dico["Dribbling"]
            if "Marking" in dico.keys():
                Marking = dico["Marking"]
            if "Aggression"in dico.keys():
                Aggression = dico["Aggression"]
            if "Reactions" in dico.keys():
                Reactions = dico["Reactions"]
            if "Interceptions"in dico.keys():
                Interceptions = dico["Interceptions"]
            if "Vision" in dico.keys():
                Vision = dico["Vision"]
            if "Composure" in dico.keys():
                Composure = dico["Composure"].split()[0]
            if "Crossing" in dico.keys():
                Crossing = dico['Crossing']
            if "Acceleration" in dico.keys():
                Acceleration = dico['Acceleration']
            if "Strength" in dico.keys():
                Strength = dico["Strength"].split()[0]
            if "Agility" in dico.keys():
                Agility = dico['Agility']
            if "Penalties" in dico.keys():
                Penalties = dico['Penalties']
            if "Volleys" in dico.keys():
                Volleys = dico["Volleys"].split()[0]
            if "Power" in dico.keys():
                Power = dico['Power']
            if "Jumping" in dico.keys():
                Jumping = dico['Jumping']
            if "Heading" in dico.keys():
                Heading = dico["Heading"].split()[0]
            if "Curve" in dico.keys():
                Curve = dico['Curve']
            if "Injury" in dico.keys():
                Injury = dico['Injury']
            if "Finishing" in dico.keys():
                Finishing = dico['Finishing']
            return Dribbling,Marking,Aggression ,Reactions ,Interceptions ,Vision ,Composure ,Crossing ,Acceleration ,Strength ,Balance ,Agility,Penalties ,Volleys ,Power ,Jumping ,Heading,Curve,Injury,player_id,Finishing
    return None

In [59]:
for index,row in df.iterrows():
    data = get_out_of_the_dico(row["dico_fifa"],row["id"])
    df_with_new_col.loc[len(df_with_new_col)]= data
    

None
90
('90', 'None', '63', '93', '55', '85', '92', '80', '89', '75', None, '90', '86', '84', None, '69', '59', '84', None, 1250, '93')
96
('96', 'None', '70', '93', '26', '93', '94', '90', '94', '84', None, '94', '88', '83', None, '86', '54', '90', None, 2371, '93')
83
('83', 'None', '80', '92', '44', '87', '92', '80', '67', '83', None, '68', '92', '87', None, '79', '85', '82', None, 647, '93')
88
('88', 'None', '75', '91', '35', '84', '84', '78', '91', '72', None, '93', '75', '75', None, '87', '84', '77', None, 838, '86')
76
('76', 'None', '86', '88', '49', '74', '86', '69', '85', '65', None, '74', '85', '85', None, '80', '78', '69', None, 755, '90')
85
('85', 'None', '85', '87', '56', '77', '80', '73', '88', '71', None, '86', '69', '79', 'Header', '81', '84', '77', None, 6854, '86')
88
('88', 'None', '77', '74', '35', '72', '81', '75', '90', '77', None, '85', '78', '68', None, '58', '47', '71', None, 522, '81')
86
('86', 'None', '59', '91', '30', '80', '77', '79', '94', '65', None,

C:\Users\regis\AppData\Local\Temp\ipykernel_8988\1530240830.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_with_new_col.loc[len(df_with_new_col)]= data
C:\Users\regis\AppData\Local\Temp\ipykernel_8988\1530240830.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_with_new_col.loc[len(df_with_new_col)]= data


60
('60', 'None', '66', '79', '79', '72', '80', '67', '41', '74', None, '48', '60', '56', None, '84', '77', '60', None, 6851, '51')
75
('75', 'None', '83', '80', '80', '87', '86', '80', '63', '67', None, '68', '76', '82', None, '66', '53', '85', None, 6853, '65')
80
('80', 'None', '69', '72', '36', '70', '70', '72', '76', '64', None, '80', '64', '69', None, '77', '72', '74', None, 6894, '75')
77
('77', 'None', '62', '68', '31', '69', '65', '75', '78', '56', None, '73', '65', '48', None, '58', '49', '76', None, 8150, '68')
75
('75', 'None', '82', '78', '76', '67', '72', '77', '79', '68', None, '78', '44', '48', None, '77', '49', '66', None, 8864, '67')
83
('83', 'None', '73', '83', '51', '88', '79', '88', '78', '58', None, '83', '67', '74', None, '33', '52', '90', None, 8992, '73')
71
('71', 'None', '76', '74', '75', '72', '73', '63', '67', '75', None, '65', '52', '70', None, '72', '55', '48', None, 9680, '67')
74
('74', 'None', '84', '86', '86', '72', '80', '54', '53', '82', None, '55'

C:\Users\regis\AppData\Local\Temp\ipykernel_8988\1530240830.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_with_new_col.loc[len(df_with_new_col)]= data
C:\Users\regis\AppData\Local\Temp\ipykernel_8988\1530240830.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_with_new_col.loc[len(df_with_new_col)]= data
C:\Users\regis\AppData\Local\Temp\ipykernel_8988\1530240830.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_with_new_col.loc[len(df_with_new_col)]= data
C:\Users\regis\AppData\Local\Temp\ipykernel_8988\1530240830.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future ver

60
('60', 'None', '73', '75', '77', '64', '74', '40', '53', '76', None, '62', '38', '29', None, '66', '76', '31', 'Prone', 6273, '27')
67
('67', 'None', '80', '75', '82', '51', '74', '38', '69', '87', None, '60', '41', '28', None, '75', '79', '32', 'Prone', 6326, '29')
53
('53', 'None', '65', '73', '80', '31', '61', '34', '54', '77', None, '47', '40', '34', None, '66', '67', '25', None, 6377, '23')
63
('63', 'None', '88', '64', '76', '59', '64', '43', '65', '70', None, '74', '45', '49', None, '69', '64', '50', None, 6418, '37')
71
('71', 'None', '75', '73', '31', '61', '74', '64', '64', '73', None, '64', '85', '71', 'Header', '80', '77', '72', 'Prone', 6477, '75')
74
('74', 'None', '76', '72', '74', '64', '70', '76', '74', '74', None, '62', '42', '55', None, '72', '71', '65', 'Prone', 6485, '62')
78
('78', 'None', '42', '65', '42', '71', '60', '67', '87', '53', None, '82', '60', '55', None, '41', '39', '66', None, 6492, '68')
75
('75', 'None', '85', '76', '75', '71', '69', '77', '74', 

C:\Users\regis\AppData\Local\Temp\ipykernel_8988\1530240830.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_with_new_col.loc[len(df_with_new_col)]= data


In [61]:
final_df = df.merge(df_with_new_col,on='id')

In [65]:
final_df.drop(columns=df.columns[:1],axis=1, inplace=True)


KeyError: "['Unnamed: 0.12'] not found in axis"

In [66]:
final_df

,Unnamed: 0.1,player_international,actual_team,number,Name,Nationalite,mois_de_naissance,jour_de_naissance,anne_de_naissance,age_actuel,...,Balance,Agility,Penalties,Volleys,Power,Jumping,Heading,Curve,Injury,Finishing
0,1,True,Liverpool FC,11,Mohamed Salah Hamed Mahrous Ghaly[1],Égyptien,juin,15,1992.0,30.0,...,None,90,86,84,None,69,59,84,None,93
1,2,True,Al-Nassr FC,7,Cristiano Ronaldo dos Santos Aveiro,Portugaise,février,5,1985.0,37.0,...,None,94,88,83,None,86,54,90,None,93
2,3,True,Tottenham Hotspur,10,Harry Edward Kane,Britannique,juillet,28,1993.0,29.0,...,None,68,92,87,None,79,85,82,None,93
3,4,True,Bayern Munich,17,NaN,Sénégalais,avril,10,1992.0,30.0,...,None,93,75,75,None,87,84,77,None,86
4,5,True,Leicester City,9,Jamie Richard Vardy,Britannique,janvier,11,1987.0,36.0,...,None,74,85,85,None,80,78,69,None,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,420,True,Burnley FC,6,Conrad Jaden Egan-Riley,Britannique Irlandais,janvier,2,2003.0,20.0,...,None,55,42,30,None,71,62,40,None,25
416,421,True,Brighton & Hove,28,Evan Joe Ferguson,Irlandais,octobre,19,2004.0,18.0,...,None,66,57,47,None,68,49,45,None,60
417,422,True,Paços de Ferreira(en prêt de Manchester City),11,Kayky da Silva Chagas,Brésilien,juin,11,2003.0,19.0,...,None,83,64,50,None,46,38,45,None,64
418,423,True,Watford FC,39,NaN,Congolais (RDC),juin,3,1998.0,24.0,...,None,76,41,55,None,82,58,62,None,61


In [67]:
final_df.to_csv('joueur_pl_fifa.csv')

# Scrap marquet

In [2]:
import pandas as pd 

In [29]:
final_df = pd.read_csv('joueur_pl_fifa.csv')

In [70]:
final_df.shape

(420, 54)

In [1]:
from selenium import webdriver
driver =  webdriver.Firefox()

In [ ]:
driver.get("https://www.transfermarkt.fr/")

In [74]:
#get the player name
#return value
def get_the_value(player_name):
    driver.get("https://www.transfermarkt.fr/")
    #get the name of the player 
    player_name = row['player_name']
    # rigth the name in the shearch_bar
    shearch_bar = driver.find_element('xpath', '/html/body/div[2]/header/div[1]/div[2]/form/input[1]')
    shearch_bar.send_keys(player_name)
    lunch = driver.find_element('xpath', '/html/body/div[2]/header/div[1]/div[2]/form/input[2]')
    # do the reschear 
    lunch.click()
    player_value = driver.find_element('xpath', '/html/body/div[2]/main/div[1]/div/div/div[1]/div/table/tbody/tr/td[6]')
    # get the data 
    data = player_value.get_attribute('innerHTML')
    return data

In [87]:
final_df['value'][200]

'32,00 mio. €'

In [91]:
for index,row in final_df.iterrows():
    if row['value']=='32,00 mio. €':
        print(index)
        data = get_the_value(row)
        final_df['value'].at[index]= data

6
17
33
44
61
170
193
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419


In [82]:
final_df['value']

0      80,00 mio. €
1      20,00 mio. €
2      90,00 mio. €
3      60,00 mio. €
4       4,00 mio. €
           ...     
415    32,00 mio. €
416    32,00 mio. €
417    32,00 mio. €
418    32,00 mio. €
419    32,00 mio. €
Name: value, Length: 420, dtype: object

In [68]:
final_df["value_fifa"]=None
final_df["value_fifa"] = final_df["value_fifa"].progress_apply(get_the_value)

  0%|          | 1/420 [00:00<05:19,  1.31it/s]


NoSuchElementException: Message: Unable to locate element: /html/body/div[2]/main/div[1]/div[2]/div/div/div[1]/a/div[1]/img
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:182:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:394:5
element.find/</<@chrome://remote/content/marionette/element.sys.mjs:275:16


In [92]:
final_df.to_csv('all_data_pl.csv')

In [3]:
import pandas as pd

In [25]:
test = pd.read_csv('df_preprocess.csv',sep=';')

In [26]:
test.columns

Index(['Unnamed: 0', 'Colonne1', 'player_international', 'number',
       'Nationalite', 'mois_de_naissance', 'jour_de_naissance',
       'anne_de_naissance', 'age_actuel', 'taille_en_m', 'Poste', 'Pied_fort',
       'id', 'player_name', 'games', 'time', 'goals', 'xG', 'assists', 'xA',
       'shots', 'key_passes', 'yellow_cards', 'red_cards', 'position',
       'team_title', 'npg', 'npxG', 'xGChain', 'xGBuildup', 'Dribbling',
       'Marking', 'Aggression', 'Reactions', 'Interceptions', 'Vision',
       'Composure', 'Crossing', 'Acceleration', 'Strength', 'Agility',
       'Penalties', 'Volleys', 'Jumping', 'Heading', 'Curve', 'Injury',
       'Finishing', 'player_value', 'dateDeNaissance,,,'],
      dtype='object')

In [16]:
test

,Unnamed: 0.1,player_international,actual_team,number,Name,Nationalite,mois_de_naissance,jour_de_naissance,anne_de_naissance,age_actuel,...,Balance,Agility,Penalties,Volleys,Power,Jumping,Heading,Curve,Injury,Finishing
0,0,True,Valence CF,7,Edinson Roberto Cavani Gómez,Uruguayen Italien,février,14,1987,35,...,NaN,64.0,84.0,86.0,Header,87.0,85.0,76.0,Prone,85.0
1,1,True,Tigres UANL,26,Florian Tristan Mariano Thauvin[1],Français,janvier,26,1993,29,...,NaN,84.0,78.0,75.0,NaN,76.0,70.0,84.0,NaN,72.0
2,2,True,FC Barcelone,14,NaN,Néerlandais,février,13,1994,28,...,NaN,80.0,76.0,75.0,NaN,69.0,65.0,85.0,NaN,84.0
3,3,True,Paris Saint-Germain,10,Neymar da Silva Santos Júnior,Brésilien,février,5,1992,30,...,NaN,93.0,91.0,86.0,NaN,62.0,63.0,88.0,Prone,83.0
4,4,True,FC Sion,45,Mario Balotelli Barwuah [1],Ghanéen Italien (depuis 2008)[2],août,12,1990,32,...,NaN,68.0,91.0,78.0,Free-Kick,72.0,72.0,76.0,Prone,80.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
386,386,True,LOSC Lille,22,Timothy Tarpeh Weah,Américain Libérien Français,février,22,2000,22,...,NaN,86.0,60.0,71.0,NaN,68.0,67.0,64.0,NaN,72.0
387,387,True,FC Sochaux-Montbéliard,4,NaN,Sénégalais,décembre,20,1993,29,...,NaN,71.0,38.0,43.0,NaN,76.0,66.0,63.0,NaN,27.0
388,388,True,UD Las Palmas,12,Enzo Lilian Marin Loiodice,Français,novembre,27,2000,22,...,NaN,71.0,58.0,39.0,NaN,63.0,58.0,67.0,NaN,51.0
389,389,True,NaN,NaN,Ambroise Oyongo Bitolo,Camerounais,juin,22,1991,31,...,NaN,85.0,59.0,45.0,NaN,72.0,42.0,69.0,Prone,54.0


In [19]:
for index,row in test.iterrows():
        row["games"] = int(row["games"])
        test.iloc[index] = row

32
35
36
20
28
30
35
38
35
31
36
26
37
32
33
37
24
31
35
29
19
29
15
29
33
38
29
34
22
26
31
36
27
32
34
28
37
32
36
29
16
24
34
14
34
30
15
14
30
25
20
36
35
15
24
31
24
33
37
26
31
29
36
34
34
32
26
22
16
34
23
22
31
29
36
14
25
26
20
18
31
25
32
31
29
37
16
13
28
23
21
26
28
20
16
30
29
28
35
28
20
30
33
26
36
26
14
16
31
24
18
22
30
19
18
16
28
25
19
21
18
33
27
26
10
13
25
33
19
5
13
20
32
29
31
25
20
20
33
20
30
33
37
20
19
14
38
21
29
32
36
16
37
12
30
11
33
18
17
12
6
28
27
13
34
22
27
9
35
22
32
25
8
6
28
30
26
12
13
3
33
3
23
27
15
30
4
37
19
31
14
10
34
8
16
3
15
1
38
21
7
21
36
12
17
15
3
8
34
33
2
3
24
4
4
2
26
10
8
15
24
28
20
29
28
14
30
14
24
29
10
35
20
2
26
1
34
21
18
15
38
3
1
13
33
23
13
35
25
5
6
12
18
1
13
4
26
35
24
37
5
14
29
22
15
28
29
22
1
17
4
1
12
28
10
28
3
34
37
19
1
24
7
19
26
26
11
4
3
23
38
36
33
26
19
8
10
29
21
2
13
31
7
33
7
19
9
25
1
14
17
29
1
5
38
28
23
3
9
18
9
3
12
18
6
1
3
7
1
4
6
13
27
37
34
16
6
3
17
11
19
14
25
17
25
21
22
21
26
8
5
17
25
3

In [22]:
type(test["games"][0])

numpy.int64